In [2]:
import datetime
import numpy as np
import pandas as pd
import os
from scipy.io import loadmat
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
# from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset,DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

# Hyperparameters

In [3]:
# hyper
features = ['SOH','voltage_measured', 'current_measured',
            'temperature_measured', 'time']
batch_size = 1  # 1*len(every_file)
input_size = len(features)
hidden_size = 128
num_layers = 1
output_size = 1
seq_len = 20   # 预测序列长度
epoch = 1000   # 1*len(train_directory)
learning_rate = 0.001  # upgrade to adaptive lr?

save_path = 'seq{}_.pth'.format(str(seq_len))  # model path
train_directory = '../datasets/train/'
val_directory = '../datasets/val/'
test_directory = '../datasets/alldataset/'

# 1.Data preprocess

## 1.1 Load Data from mat

In [4]:
def load_data(battery_path):
    # mat = loadmat('../datasets/BatteryDataset/' + battery_id + '.mat') # MAT PATH ESSENTIAL origin
    battery_id = battery_path[-9:-4]
    mat = loadmat(battery_path) # MAT PATH ESSENTIAL

    counter = 0
    dataset = []
    capacity_data = []
    # print('Total data in dataset: ', len(mat[battery][0, 0]['cycle'][0])) #total cycle

    for i in range(len(mat[battery_id][0, 0]['cycle'][0])):
        row = mat[battery_id][0, 0]['cycle'][0, i]
        if row['type'][0] == 'discharge':
            ambient_temperature = row['ambient_temperature'][0][0]
            date_time = datetime.datetime(int(row['time'][0][0]),
                                          int(row['time'][0][1]),
                                          int(row['time'][0][2]),
                                          int(row['time'][0][3]),
                                          int(row['time'][0][4])) + datetime.timedelta(seconds=int(row['time'][0][5]))
            data = row['data']
            capacity = data[0][0]['Capacity'][0][0]
            for j in range(len(data[0][0]['Voltage_measured'][0])):
                voltage_measured = data[0][0]['Voltage_measured'][0][j]
                current_measured = data[0][0]['Current_measured'][0][j]
                temperature_measured = data[0][0]['Temperature_measured'][0][j]
                current_load = data[0][0]['Current_load'][0][j]
                voltage_load = data[0][0]['Voltage_load'][0][j]
                time = data[0][0]['Time'][0][j]
                dataset.append([counter + 1, ambient_temperature, date_time, capacity,
                                voltage_measured, current_measured,temperature_measured,
                                current_load, voltage_load, time])

            capacity_data.append([counter + 1, ambient_temperature, date_time, capacity,
                                  voltage_measured, current_measured,temperature_measured,
                                  current_load, voltage_load, time])
            counter = counter + 1
    # print(dataset[0])

    return [pd.DataFrame(data=dataset,
                         columns=['cycle', 'ambient_temperature', 'datetime','capacity',
                                  'voltage_measured','current_measured', 'temperature_measured',
                                  'current_load', 'voltage_load', 'time']),
            pd.DataFrame(data=capacity_data,
                         columns=['cycle', 'ambient_temperature', 'datetime','capacity',
                                  'voltage_measured','current_measured', 'temperature_measured',
                                  'current_load', 'voltage_load', 'time'])]


# test_battery_id0 = 'B0050'
# test_battery_id1 = 'B0052'
# test_battery_id2 = 'B0055'
# test_battery_id3 = 'B0056'
# dataset0, capacity0 = load_data(test_directory+ test_battery_id0 + '.mat')
# dataset1, capacity1 = load_data(test_directory+ test_battery_id1 + '.mat')
# dataset2, capacity2 = load_data(test_directory+ test_battery_id2 + '.mat')
# dataset3, capacity3 = load_data(test_directory+ test_battery_id3 + '.mat')
# # capacity.to_csv('../datasets/BatteryCSV/{}.csv'.format(test_battery_id), index=False)
# # pd.set_option('display.max_columns', 10)
# # print(capacity.head(5))
# # dataset.describe()
#
# # 创建一个包含两个子图的 Figure 对象
# fig, axs = plt.subplots(2, 2, figsize=(8, 6))
# sns.set_style("darkgrid") # 黑色的背景，带有白色网格线
# # 在第一个子图中绘制 Sin(x)
# axs[0, 0].plot(capacity0['cycle'], capacity0['capacity'])
# axs[0, 0].set_title('Discharge {}'.format(test_battery_id0))
# axs[0, 0].set_xlabel('cycle')
# axs[0, 0].set_xlabel('Capacity')
# # 在第二个子图中绘制 Cos(x)
# axs[0, 1].plot(capacity1['cycle'], capacity1['capacity'])
# axs[0, 1].set_title('Discharge {}'.format(test_battery_id1))
# axs[0, 1].set_xlabel('cycle')
# axs[0, 1].set_xlabel('Capacity')
#
# # 在第三个子图中绘制 Cos(x)
# axs[1, 0].plot(capacity2['cycle'], capacity2['capacity'])
# axs[1, 0].set_title('Discharge {}'.format(test_battery_id2))
# axs[1, 0].set_xlabel('cycle')
# axs[1, 0].set_xlabel('Capacity')
#
# # 在第四个子图中绘制 Cos(x)
# axs[1, 1].plot(capacity3['cycle'], capacity3['capacity'])
# axs[1, 1].set_title('Discharge {}'.format(test_battery_id3))
# axs[1, 1].set_xlabel('cycle')
# axs[1, 1].set_xlabel('Capacity')
#
# # 调整布局
# plt.tight_layout()
# # 显示图像
# plt.show()


## 1.2 normlize data

In [5]:
def norm_data(battery_id):
    dataset, capacity = load_data(battery_id)
    if capacity['capacity'][0] > capacity['capacity'][1] :
        C = capacity['capacity'][0]
    else :
        C = capacity['capacity'][1]
    soh = []
    for i in range(len(capacity)):
        soh.append([capacity['capacity'][i] / C])
    soh = pd.DataFrame(data=soh, columns=['SOH'])

    # features for training
    attribs=['capacity', 'voltage_measured', 'current_measured',
             'temperature_measured', 'current_load', 'voltage_load', 'time']
    train_dataset = capacity[attribs]
    sc = MinMaxScaler(feature_range=(0,1)) # = (num-min)/(max-min)
    train_dataset = sc.fit_transform(train_dataset) # issue：not based on Rated
    # print(train_dataset.shape)
    # print(soh.shape)
    attribs_scaled = pd.DataFrame(data=train_dataset,columns=attribs)
    return  pd.concat([capacity['cycle'], attribs_scaled, soh], axis=1)

def data_loader(battery_id, seq_len, features):
    dataset = norm_data(battery_id)
    input_size = len(features)
    data_set_train=dataset[features].values
    x_train=[]
    label=[]
    batch = len(data_set_train)
    #take the last 10t to predict 10t+1
    for i in range (seq_len,batch):
        x_train.append(data_set_train[i-seq_len:i,:])
        label.append(data_set_train[i,0])

    x_train = np.array(x_train)
    x_train = np.reshape(x_train,(batch-seq_len,seq_len,input_size)) #(batch,seq_len,input_size)
    x_train = torch.tensor(x_train, dtype=torch.float32)
    label = torch.tensor(label, dtype=torch.float32).view(-1, 1) # (batch,seq_len)取最后一个输出
    return x_train, label

# # storge
# features = ['SOH','voltage_measured', 'current_measured',
#             'temperature_measured', 'current_load', 'voltage_load', 'time']
# train_data,train_data_real = data_loader('B0005', 10 , features)
# train_data,train_data_real
# dataset_scaled.to_csv('../datasets/BatteryCSV/B0005norm.csv', index=False)

## 1.3 dataloader

In [6]:
class LoadDataset(Dataset):
    def __init__(self, root_dir, seq_len, features,  transform=None):
        self.root_dir = root_dir
        self.seq_len  = seq_len
        self.features = features
        self.transform = transform
        self.file_list = sorted(os.listdir(root_dir))  # Assumes file names determine order

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        battery_id = self.file_list[idx]
        battery_path = os.path.join(self.root_dir, battery_id)
        dataset, label = data_loader(battery_path, self.seq_len, self.features)
        # if self.transform:
        #     dataset = self.transform(dataset)
        #     label = self.transform(label)

        return dataset, label


# # Example usage
# custom_dataset = LoadDataset(train_directory, seq_len=seq_len, features=features)
#
# # Create DataLoader
# batch_size = 1
# dataloader = DataLoader(custom_dataset, batch_size=batch_size)
#
# for train_data, train_data_real in dataloader:
#     # train_data=torch.squeeze(train_data)
#     print(train_data.shape)

## 2. training
class torch.nn.LSTM(*args, **kwargs)
参数有：
    input_size：x的特征维度
    hidden_size：隐藏层的特征维度
    num_layers：lstm隐层的层数，默认为1
    bias：False则bihbih=0和bhhbhh=0. 默认为True
    batch_first：True则输入输出的数据格式为 (batch, seq_len, feature)
    dropout：除最后一层，每一层的输出都进行dropout，默认为: 0
    bidirectional：True则为双向lstm默认为False

LSTM input(seq_len, batch, input_size)
参数有：
    batch：每次喂给网络的数据条数，在NLP中就是一次喂给网络多少个句子 !!!
    seq_len：序列长度，在NLP中就是句子长度，一般都会用pad_sequence补齐长度!!!
    input_size：特征维度，和前面定义网络结构的input_size一致。

output,(ht, ct) = net(input)
    output: 最后一个状态的隐藏层的神经元输出 size = （seq_Len, batch, num_directions * hidden_size)
    ht：最后一个状态的隐含层的状态值
    ct：最后一个状态的隐含层的遗忘门值



In [7]:
# net structure
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm1 = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True)
        self.lstm3 = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True)
        self.lstm4 = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):

        out, _ = self.lstm1(x) # shape（seq_Len, batch, num_directions * hidden_size)
        out, _ = self.lstm2(out)
        out, _ = self.lstm3(out)
        out, _ = self.lstm4(out)

        out = self.fc(out[:, -1, :])  # 使用最后一个时间步的输出的输入（seq_Len, batch, num_directions * hidden_size)
        # print(out.shape)
        return out


In [32]:
# train

# checking if GPU is available
device = torch.device("cpu")
if (torch.cuda.is_available()):
    device = torch.device("cuda:0")
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

# 构建模型
model = LSTMModel(input_size=input_size,
                  hidden_size=hidden_size,
                  output_size=output_size,
                  num_layers = num_layers)
model = model.to(device) # lstm doesnt work on gpu?
# train
train_dataset = LoadDataset(train_directory, seq_len=seq_len, features=features)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size)

# validation
val_dataset = LoadDataset(val_directory, seq_len=seq_len, features=features)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

# 定义损失函数和优化器
criterion = nn.MSELoss()
# criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 储存路径
work_dir = './LSTM'
# 添加tensorboard
writer = SummaryWriter("{}/logs".format(work_dir))

# model = model.to(device)
criterion = criterion.to(device)

# checkponits = epoch // 5

# 训练模型
for epoch in range(epoch):
    print("-------epoch  {} -------".format(epoch))
    # 训练步骤
    model.train()
    total_train_loss = 0
    for train_data, train_data_real in train_dataloader:
        train_data = torch.squeeze(train_data).to(device)
        train_data_real = torch.squeeze(train_data_real).to(device)

        output = model(train_data)
        output = torch.squeeze(output)
        optimizer.zero_grad()
        train_loss = criterion(output, train_data_real)
        train_loss.backward()
        optimizer.step()

        total_train_loss += train_loss.item()
    avg_train_loss = total_train_loss/len(train_dataloader)
    print("train set Loss: {}".format(avg_train_loss)) # 出现nan可能是seq_len太长了,有些数据集比seq_len短

    # 测试步骤
    model.eval()
    total_val_loss = 0
    with torch.no_grad():#用于在推断或验证阶段，当不需要计算梯度时，以提高效率和减少内存占用
        for val_data, val_data_real in val_dataloader:
            val_data = torch.squeeze(val_data).to(device)
            val_data_real = torch.squeeze(val_data_real).to(device)

            val_output = model(val_data)
            val_output = torch.squeeze(val_output)
            val_loss = criterion(val_output, val_data_real)
            total_val_loss += val_loss.item()

    avg_val_loss = total_val_loss/len(val_dataloader)
    print("val set Loss: {}".format(avg_val_loss))

    # save checkpoint
    # if epochs % checkponits == 0:
    if avg_train_loss < 0.0001  and avg_val_loss.item()< 0.0001 :
        torch.save(model.state_dict(), str('EX_')+save_path[:-4]+str(epoch)+'.pth')
        print("save model:epoch {}".format(epoch))
    if avg_train_loss < 0.0005  and avg_val_loss.item()< 0.0005 :
        torch.save(model.state_dict(), str('A_')+save_path[:-4]+str(epoch)+'.pth')
        print("save model:epoch {}".format(epoch))
    if avg_train_loss < 0.001  and avg_val_loss.item()< 0.001 :
        torch.save(model.state_dict(), str('B_')+save_path[:-4]+str(epoch)+'.pth')
        print("save model:epoch {}".format(epoch))
# save last1
torch.save(model.state_dict(), save_path[:-4]+'last.pth')

Training on GPU.
-------epoch  0 -------
train set Loss: 0.2544890487327939
val set Loss: 0.16656597455342612
-------epoch  1 -------
train set Loss: 0.22176036112010478
val set Loss: 0.035358903308709465
-------epoch  2 -------
train set Loss: 0.16206913174130022
val set Loss: 0.029704665765166283
-------epoch  3 -------
train set Loss: 0.1388199324347079
val set Loss: 0.07080391049385071
-------epoch  4 -------
train set Loss: 0.10440354688558727
val set Loss: 0.04578728104631106
-------epoch  5 -------
train set Loss: 0.07244299510493875
val set Loss: 0.022380041579405468
-------epoch  6 -------
train set Loss: 0.09303417288232595
val set Loss: 0.05826391854013006
-------epoch  7 -------
train set Loss: 0.04013777613872662
val set Loss: 0.01315513210526357
-------epoch  8 -------
train set Loss: 0.03277038873173296
val set Loss: 0.013117047453609606
-------epoch  9 -------
train set Loss: 0.026097404191241368
val set Loss: 0.007776088508156438
-------epoch  10 -------
train set Loss

# 3.eval and plot comparation

In [9]:
test_battery_id = 'B0018'
# save_path = save_path
save_path = 'seq20_last.pth'

test_battery_path = test_directory+ test_battery_id + '.mat'
test_data,test_data_real = data_loader(test_battery_path,seq_len,features)
# load
lstm = LSTMModel(input_size=input_size, hidden_size=hidden_size, output_size=output_size, num_layers=num_layers)
lstm.load_state_dict(torch.load(save_path))
lstm.eval()
output = lstm(test_data)
predicted_data = output.detach().numpy().reshape(-1)
print(predicted_data.shape)

# draw
origin_data = norm_data(test_battery_path) # dictionary
sns.set_style("white")
plt.figure(figsize=(8, 5))
#Draw
plt.plot([i for i in range(len(origin_data['SOH']))], origin_data['SOH'], label='SOH')
# predicted_data = np.concatenate((origin_data['SOH'][0:seq_len],predicted_data), axis = 0)
plt.plot([i for i in range(seq_len,len(origin_data['SOH']))], predicted_data, label='Predicted SOH')
plt.plot([0.,len(origin_data['SOH'])], [0.70, 0.70], label='Threshold')

# make x-axis ticks legible
adf = plt.gca().get_xaxis().get_major_formatter()
plt.xlabel('cycle')
plt.ylabel('SOH')
plt.title('Discharge {}'.format(test_battery_id))


MAE =  np.mean(np.abs(predicted_data - origin_data['SOH'][seq_len:]))
MAPE = np.mean(np.abs((origin_data['SOH'][seq_len:] - predicted_data) / origin_data['SOH'][seq_len:]))
MSE = np.mean((predicted_data - origin_data['SOH'][seq_len:]) ** 2)
RMSE = np.sqrt(MSE)
print('MAE:{}'.format(MAE))
print('MAPE:{}'.format(MAPE))
print('MSE:{}'.format(MSE))
print('RMSE:{}'.format(RMSE))

FileNotFoundError: [Errno 2] No such file or directory: 'seq20_last.pth'